In [1]:
# ! gdown 1qhn9-RgkfFHIGD7pDGIvwjQehGKM-5Bi

In [2]:
from textblob import TextBlob

In [3]:
import pandas as pd
df = pd.read_csv('./scraping/ftx.csv')

In [4]:
df

,Datetime,Tweet Id,Text,Username,Followers
0,1.669853e+09,1598104583779516417,@MarioNawfal @SBF_FTX When you headed to priso...,mnboy1989,76
1,1.669853e+09,1598104576112332800,"'I didn't ever try to commit fraud on anyone,'...",ftxtracktbot,5
2,1.669853e+09,1598104573398360064,How CZ helps a friend SAM (FTX) https://t.co/Z...,cryptocapitalto,18
3,1.669853e+09,1598104567845355520,@BrynneKKelly And as Joe average gets his gold...,7h0ras1c,99
4,1.669853e+09,1598104567639465984,@BrynneKKelly And as Joe average gets his gold...,7h0ras1c,99
...,...,...,...,...,...
355602,1.668956e+09,1594342047581732869,Institutional trading firm Genesis announced o...,PremSag24970777,2
355603,1.668956e+09,1594342035649335298,"Криптовалютная торговая площадка FTX объявила,...",bits_media,6109
355604,1.668956e+09,1594342029957685249,@JamesBradleyCA thought it was up to the peopl...,TheScot60,45
355605,1.668956e+09,1594342028522835968,吴说获悉，根据披露的债权人文件，FTX 集团前五十大债权人的未担保债权达到约 30 亿美金，...,wublockchain12,75277


In [5]:
import re
import nltk
from nltk import PorterStemmer
ps = PorterStemmer()
nltk.download("stopwords")
from nltk.corpus import stopwords
stop = stopwords.words('english')
def cleantweets(txt):
  # Remove mentions
  txt = re.sub(r"@","", txt)
  # Remove hashtags
  txt = re.sub(r"#","", txt)
  # Remove urls
  txt = re.sub(r"https?:\/\/[ A−Za−z0 −9\.\/]+", "", txt)
  # Remove Stop Words
  txt = ' '.join([word for word in txt.split() if word not in (stop)])
  #Stemming
  txt = ' '.join([ps.stem(word) for word in txt.split()])
  return txt

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/piyushk/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
def getsubjectivity(txt):
  return TextBlob(txt).sentiment.subjectivity

In [7]:
def getpolarity(txt):
  return TextBlob(txt).sentiment.polarity

In [8]:
def getanalysis(a):
  if a < 0:
    return "Negative"
  elif a == 0:
    return "Neutral"
  else:
    return "Positive"

In [9]:
def prepare_twitter_data(tweet_data):
  results = tweet_data.copy()
  results["clean text"] = results["Text"].apply(cleantweets)
  results["polarity"] = results["Text"].apply(getpolarity)
  results["subjectivity"] = results["Text"].apply(getsubjectivity)
  results["tweet_class"] = results["polarity"].apply(getanalysis)
  return results

In [11]:
processed_tweets = prepare_twitter_data(df)

In [ ]:
from langdetect import detect

df = pd.DataFrame(
    columns=[
        "Datetime",
        "Tweet Id",
        "Text",
        "Username",
        "Followers",
        "clean text",
        "polarity",
        "subjectivity",
        "tweet_class",
    ]
)
from contextlib import suppress
import multiprocessing

def filter_en_text(i, x):
    with suppress(Exception):
        return detect(x) == "en"
with multiprocessing.Pool() as pool:
    # Use the pool to apply the filter function to each element in parallel
    results = pool.starmap(filter_en_text, enumerate(processed_tweets["clean text"].values))
    # Filter the original DataFrame using the results
    results = [False if v is None else v for v in results]
    df = processed_tweets[results]

In [ ]:
processed_tweets.to_csv('[ftx] processed_tweets.csv')